In [2]:
import sys
import os
import json
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from data_manager import *
from metadata import *
from preprocess_utilities import *
from data_manager import DataManager
from annotation_utilities import *

In [52]:
def shift_and_save_annotations(stack, hand_annotations_prep2, fn, prep=2):
    hdf_root = os.path.join(ROOT_DIR,'CSHL_labelings_v3',stack+'/')
    new_fn = fn[:fn.index('.hdf')]
    
    for i in range(num_annotations):
        if prep==2:
            offsets = {'MD585':[952,-448],'MD589':[-1570,-350],'MD594':[-2225,-400]}
            vertices = hand_annotations_prep2['vertices'][i]
            label_position = hand_annotations_prep2['label_position'][i]
            hand_annotations_prep2['vertices'][i] = vertices + offsets[stack]
            hand_annotations_prep2['label_position'][i] = label_position + offsets[stack]
    new_fn = hdf_root+new_fn+'_prep2_corr1.hdf'
    
    save_hdf_v2(hand_annotations_prep2, new_fn)

In [51]:
stack = "MD585"

hdf_root = ROOT_DIR+'/CSHL_labelings_v3/'+stack+'/'
if stack=="MD585":
    fn = 'MD585_annotation_contours_05312018041032.hdf'
if stack=="MD589":
    fn = 'MD589_annotation_contours_05312018031718.hdf'
if stack=="MD594":
    fn = 'MD594_annotation_contours_05312018035134.hdf'
hand_annotations = load_hdf_v2(hdf_root+fn)

num_annotations = len(hand_annotations)
    
hand_annotations_prep2 = convert_annotation_v3_original_to_aligned_cropped_v2(\
                                        hand_annotations, stack=stack,\
                                        out_resolution='raw', prep_id=2)

hand_annotations_prep5 = convert_annotation_v3_original_to_aligned_cropped_v2(\
                                        hand_annotations, stack=stack,\
                                        out_resolution='raw', prep_id=5)

Seems you are using operation INIs to provide cropbox.
Seems you are using operation INIs to provide cropbox.


In [ ]:
shift_and_save_annotations(stack, hand_annotations_prep2, fn, prep=2)

In [8]:
section = 295
prep = 5

imgtype = 'raw'

fn = metadata_cache['sections_to_filenames'][stack][section]
print fn

img_fp = os.environ['ROOT_DIR']+'CSHL_data_processed/'+stack+'/'+stack+'_prep'+str(prep)+'_'+imgtype+'_gray/'+fn+'_prep'+str(prep)+'_'+imgtype+'_gray.tif'
img = imread(img_fp)

MD589-IHC57-2015.07.31-01.18.16_MD589_1_0169


IOError: [Errno 2] No such file or directory: u'/media/alexn/BstemAtlasDataBackup/ucsd_brain/CSHL_data_processed/MD589/MD589_prep5_raw_gray/MD589-IHC57-2015.07.31-01.18.16_MD589_1_0169_prep5_raw_gray.tif'

In [25]:
print stack
print ''

rostral_lim, caudal_lim, dorsal_lim, ventral_lim = DataManager.load_cropbox_v2(stack, prep_id=2, only_2d=True)
print 'PREP 2'
print [rostral_lim, caudal_lim, dorsal_lim, ventral_lim]
print 'width', caudal_lim-rostral_lim
print 'height', ventral_lim-dorsal_lim
print 'width', (caudal_lim-rostral_lim)*32
print 'height', (ventral_lim-dorsal_lim)*32
print ''

rostral_lim, caudal_lim, dorsal_lim, ventral_lim = DataManager.load_cropbox_v2(stack, prep_id=5, only_2d=True)
print 'PREP 5'
print [rostral_lim, caudal_lim, dorsal_lim, ventral_lim]
print 'width', caudal_lim-rostral_lim
print 'height', ventral_lim-dorsal_lim
print 'width', (caudal_lim-rostral_lim)*32
print 'height', (ventral_lim-dorsal_lim)*32

MD585

PREP 2
[521, 1057, 128, 465]
width 536
height 337
width 17152
height 10784

PREP 5
[50, 1188, 21, 738]
width 1138
height 717
width 36416
height 22944


Seems you are using operation INIs to provide cropbox.
Seems you are using operation INIs to provide cropbox.


In [10]:
np.shape(img)

(22976, 36448)

In [20]:
plt.figure(figsize=(20, 20))
plt.imshow(img, cmap='gray')

# Prep 2
offsets = {'MD585':[952,-448],'MD589':[-1570,-350],'MD594':[-2225,-400]}
# Prep 5
#prep5_to_prep2_offsets = {'MD585':[15072,3424],'MD589':[-0,-0],'MD594':[-0,-0]}
if prep==5:
    offsets = {'MD585':[0,0],'MD589':[0,0],'MD594':[0,0]}

# for structure in ['7n','IC','SNC','SNR']:
for structure in structures_unsided_sorted_by_rostral_caudal_position:
    x_offset = offsets[stack][0]
    y_offset = offsets[stack][1]
    x_raw = []
    y_raw = []
    x_thumb = []
    y_thumb = []
    
    try:
        for i in range(num_annotations):
            if prep==2:
                curr_section = hand_annotations_prep2['section'][i]
                if curr_section != section:
                    continue
                structure = hand_annotations_prep2['name'][i]
                side = hand_annotations_prep2['side'][i]
                vertices = hand_annotations_prep2['vertices'][i]
            elif prep==5:
                curr_section = hand_annotations_prep5['section'][i]
                if curr_section != section:
                    continue
                structure = hand_annotations_prep5['name'][i]
                side = hand_annotations_prep5['side'][i]
                vertices = hand_annotations_prep5['vertices'][i]
            
            for x, y in vertices:
                x_raw.append( int(round(float(x))+x_offset) )
                y_raw.append( int(round(float(y))+y_offset) )
                x_thumb.append( (float(x)/32)+x_offset/32 )
                y_thumb.append( (float(y)/32)+y_offset/32 )
    except Exception as e:
#         print structure+' skipped'
#         print e
        continue

    plt.scatter(x_raw, y_raw, s=1, c='blue', alpha=.5)

NameError: name 'img' is not defined

In [21]:
# Prep5 -> Prep2 real offset
# MD585
# prep2_thumb_real = [338, 537]  # The actual prep2 image files are this big
# prep2_thumb_cropbox_dimensions = [337, 536]  # Size in the metadata cropbox
# [15072.  3424.]
# [471. 107.]

# MD589
# prep2_thumb_real = [373, 485]
# prep2_thumb_cropbox_dimensions = [368, 484] 
# [16672.  4128.]
# [521. 129.]

# MD594
# prep2_thumb_real = [347, 538]
# prep2_thumb_cropbox_dimensions =  [348, 438]
# [17568.  2464.]
# [549.  77.]

print (hand_annotations_prep5['vertices'][0]-hand_annotations_prep2['vertices'][0])[0]
print (hand_annotations_prep5['vertices'][0]-hand_annotations_prep2['vertices'][0])[0]/32

[16672.  4128.]
[521. 129.]


In [17]:
stack

'MD594'

In [126]:
prep_id=2

xmin_down32, _, ymin_down32, _ = DataManager.load_cropbox_v2(stack, prep_id=prep_id, only_2d=True)
Ts_rawResol = DataManager.load_transforms(stack=stack, resolution='raw', use_inverse=True)

for cnt_id, cnt in hand_annotations.iterrows():
    sec = cnt['section']
    if sec not in metadata_cache['valid_sections'][stack]:
        continue
    fn = metadata_cache['sections_to_filenames'][stack][sec]
    hand_annotations.loc[cnt_id, 'section'] = sec
    
    Tinv_rawResol = Ts_rawResol[fn]

    vertices_wrt_alignedCropped_rawResol = \
    np.dot(Tinv_rawResol, np.c_[cnt['vertices'], np.ones((len(cnt['vertices']),))].T).T[:, :2] - (xmin_down32 * 32., ymin_down32 * 32.)
        
    vertices_wrt_alignedCropped_outResol = vertices_wrt_alignedCropped_rawResol
        
    hand_annotations.set_value(cnt_id, 'vertices', vertices_wrt_alignedCropped_outResol)
    hand_annotations.set_value(cnt_id, 'resolution', 'raw')

    if 'label_position' in cnt and cnt['label_position'] is not None:
        label_position_wrt_alignedCropped_rawResol = np.dot(Tinv_rawResol, np.r_[cnt['label_position'], 1])[:2] - (xmin_down32 * 32., ymin_down32 * 32.)
        
        label_position_wrt_alignedCropped_outResol = label_position_wrt_alignedCropped_rawResol
            
        hand_annotations.set_value(cnt_id, 'label_position', label_position_wrt_alignedCropped_outResol)


/media/alexn/BstemAtlasDataBackup/ucsd_brain/CSHL_data_processed/MD589/MD589_cropbox.ini


Seems you are using operation INIs to provide cropbox.


In [128]:
print hand_annotations['vertices'][80][0]

print hand_annotations_prep2['vertices'][80][0]

[7933.91304348 4182.26086957]
[7933.91304348 4182.26086957]


In [61]:
fn = metadata_cache['sections_to_filenames'][stack][section]
print fn

MD585-IHC47-2015.08.19-14.53.04_MD585_3_0141


In [28]:
! ls /media/alexn/BstemAtlasDataBackup/ucsd_brain/CSHL_data_processed/MD585/

MD585_anchor.txt		       MD585_prep2_thumbnail_gray
MD585_custom_transforms		       MD585_prep5_raw_gray
MD585_elastix_output		       MD585_prep5_thumbnail_gray
MD585_holding_cell_prep1_masks	       MD585_raw
MD585_intensity_normalization_results  MD585_raw_gray
MD585_metadata_cache.json	       MD585_raw_mask
MD585_original_image_crop.csv	       MD585_sorted_filenames.txt
MD585_prep1_thumbnail_gray	       MD585_thumbnail_gray
MD585_prep1_thumbnail_mask	       MD585_thumbnail_mask
MD585_prep2_raw_gray		       MD585_transforms_to_anchor.csv
MD585_prep2_raw_grayJpeg	       operation_configs
MD585_prep2_sectionLimits.ini


In [129]:
with open('/home/alexn/Desktop/atlas_patch_files/hand_annotated_patches/'+stack+'_patch_locations.json','r') as f:
    data=json.load(f)

In [26]:
x_raw

[7605.0,
 7453.0,
 7364.0,
 7298.0,
 7335.0,
 7405.0,
 7484.0,
 7578.0,
 7689.0,
 7833.0,
 7946.0,
 8028.0,
 8038.0,
 7993.0,
 7923.0,
 7828.0,
 7709.0,
 5589.0,
 5445.0,
 5340.0,
 5258.0,
 5184.0,
 5118.0,
 5052.0,
 4986.0,
 4920.0,
 4846.0,
 4772.0,
 4706.0,
 4648.0,
 4614.0,
 4610.0,
 4640.0,
 4677.0,
 4715.0,
 4760.0,
 4806.0,
 4958.0,
 5032.0,
 5098.0,
 5148.0,
 5214.0,
 5288.0,
 5354.0,
 5428.0,
 5494.0,
 5544.0,
 5595.0,
 5653.0,
 5719.0,
 5777.0,
 5835.0,
 5886.0,
 5913.0,
 5859.0,
 5772.0,
 5661.0,
 6792.0,
 6656.0,
 6543.0,
 6453.0,
 6372.0,
 6290.0,
 6240.0,
 6213.0,
 6217.0,
 6239.0,
 6268.0,
 6313.0,
 6367.0,
 6445.0,
 6524.0,
 6618.0,
 6714.0,
 6811.0,
 6893.0,
 6959.0,
 7025.0,
 7083.0,
 7149.0,
 7207.0,
 7242.0,
 7261.0,
 7265.0,
 7202.0,
 7124.0,
 7030.0,
 6927.0,
 6824.0,
 7686.0,
 7541.0,
 7428.0,
 7339.0,
 7312.0,
 7349.0,
 7436.0,
 7571.0,
 7692.0,
 7790.0,
 7871.0,
 7890.0,
 7828.0,
 7725.0,
 2413.0,
 2326.0,
 2239.0,
 2152.0,
 2100.0,
 2065.0,
 2030.0,
 2030.0,
 

In [72]:
metadata_cache['section_limits']['MD585']

[105, 360]

In [ ]:
del img